# Remote Sensing Advanced Methods - 2021


## So2Sat LCZ42

By 2050, Berlin summers could be as hot as in Canberra, Australia. Pankow, a district in the city’s north, has already declared a climate emergency in 2019 and is planning ahead. It is planting trees from the Mediterranean that can withstand the heat, and has calculated computer simulations for sunshine and cold air corridors for the construction of 1200 new apartments. A few changes, like swapping asphalt and concrete that store heat against greenery that soaks up water and provides shade, can make a difference on the local scale. Many of these changes on a local scale then make a difference on the bigger scale.

To understand local climate in cities, scientists have developed the Local Climate Zone classification scheme, as part of the So2Sat project. The aim is to create a 4D urban map of the world.

It differentiates between 17 zones based mainly on surface structures (such as building and tree density) as well as surface cover (green, pervious soils versus impervious grey surfaces). There are algorithms that calculate these maps from freely available satellite imagery, but there’s still room for improvement by adapting or developing suitable and advanced Convolutional Neural Network (CNN) architectures that generalise well.

The outcome of So2Sat will be the first and unique global and consistent spatial data set on urban morphology (3D/4D) of settlements, and a multidisciplinary application derivate assessing population density. This is seen as a giant leap for urban geography research as well as for formation of opinions for stakeholders based on resilient data.

```
@article{zhu2020so2sat,
  title={So2Sat LCZ42: a benchmark data set for the classification of global local climate zones [Software and Data Sets]},
  author={Zhu, Xiao Xiang and Hu, Jingliang and Qiu, Chunping and Shi, Yilei and Kang, Jian and Mou, Lichao and Bagheri, Hossein and Haberle, Matthias and Hua, Yuansheng and Huang, Rong and others},
  journal={IEEE Geoscience and Remote Sensing Magazine},
  volume={8},
  number={3},
  pages={76--89},
  year={2020},
  publisher={IEEE}
}
```  

## Agenda

1. [Meet the Data](#1.-Meet-the-Data)
2. [*Classical* Machine Learning: Random Forest Classifier](#2.-Classical-Machine-Learning:-Random-Forest-Classifier)
3. [Deep Learning](#3.-Deep-Learning)

## 1. Meet the Data

The *So2Sat LCZ42* data set is an *Earth observation* image classification data set. It contains co-registered image patches from Sentinel-1 (10 multi-spectral bands) and Sentinel-2 (8 bands) satellite sensors, all assigned to one of the 17 *local climate zones* (LCZ) classes.

The LCZ classes are as follows: 
1) compact high-rise, 
2) compact mid-rise, 
3) compact low-rise,
4) open high-rise,
5) open mid-rise,
6) open low-rise,
7) lightweight low-rise,
8) large low-rise,
9) sparsely built, 
10) heavy industry,
11) dense trees,
12) scattered tree, bush, scrub,
14) low plants,
15) bare rock or paved,
16) bare soil or sand, and 
17) water (17) 

The data set is split into training (352,366 images), validation (24,188) and test (24,119).

It is important to note that two various pools of cities were used to build So2Sat LCZ42. 32 cities around the globe were selected to form the training set, while samples from 10 different cites were used for the validation and test set, with a geographical split (east and west).

In [ ]:
# install the required packages
import sys
!{sys.executable} -m pip install -r requirements.txt

Load the required packages

In [ ]:
import h5py
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score

import tensorflow as tf

In [ ]:
# this should match the name of the downloaded data set
filename = 'data/subset_lcz42.h5'

dataset = h5py.File(filename, 'r')

# show the content names
print(list(dataset.keys()))

In [ ]:
# load the labels
labels = np.array(dataset['label'])

# show the shape
print("Labels shape: " + str(labels.shape))

# print the labels
print(labels[0,:])

In [ ]:
# load Sentinel-1 data
sen1 = np.array(dataset['sen1'])

print("Sentinel-1 shape: " + str(sen1.shape))

def false_color(X):
    """ False color visualization
    
    Sentinel-1 data in So2Sat LCZ42
        1) the real part of the unfiltered VH channel
        2) the imaginary part of the unfiltered VH channel
        3) the real part of the unfiltered VV channel
        4) the imaginary part of the unfiltered VV channel
        5) the intensity of the refined LEE filtered VH channel
        6) the intensity of the refined LEE filtered VV channel
        7) the real part of the refined LEE filtered covariance matrix off-diagonal element
        8) the imaginary part of the refined LEE filtered covariance matrix off-diagonal element
    """
    band1 = X[:,:,0]
    band2 = X[:,:,2]
    band3 = X[:,:,2]

    band1 = band1 / (band1.max()/255.0)
    band2 = band2 / (band2.max()/255.0)
    band3 = band3 / (band3.max()/255.0)

    tc = np.dstack((band1, band2, band3))
    
    return tc.astype('uint8')

# show one patch
plt.subplot(121)
# plt.imshow(10 * np.log10(sen1[10,:,:,0]), cmap=plt.cm.get_cmap('gray'))
plt.imshow(false_color(sen1[10,:,:,:]))
plt.colorbar()
plt.title('Sentinel-1')

plt.show()

In [ ]:
# load Sentinel-2 data
sen2 = np.array(dataset['sen2'])

print("Sentinel-2 shape: " + str(sen2.shape))

def true_color(X):
    """ Define True Color Sentinel image
    
    The function returns the MinMax scaled RGB bands
    
    Sentinel-2 Bands in So2Sat LCZ 42
        1) Band B2 (Blue), 10m GSD
        2) Band B3 (Green), 10m GSD
        3) Band B4 (Red), 10m GSD
        4) Band B5, upsampled to 10m from 20m GSD
        5) Band B6, upsampled to 10m from 20m GSD
        6) Band B7, upsampled to 10m from 20m GSD
        7) Band B8, 10m GSD
        8) Band B8a, upsampled to 10m from 20m GSD
        9) Band B11, upsampled to 10m from 20m GSD
        10) and Band B12, upsampled to 10m from 20m GSD

    Matplot convention RGB [0, 255]    
    """    
    blue = X[:,:,0] / (X[:,:,0].max()/255.0)
    green = X[:,:,1] / (X[:,:,1].max()/255.0)
    red = X[:,:,2] / (X[:,:,2].max()/255.0)
    
    tc = np.dstack((red, green, blue))     
    
    return tc.astype('uint8')

# show one patch
plt.subplot(122)
plt.imshow(true_color(sen2[10,:,:,0:3]))
plt.colorbar()
plt.title('Sentinel-2')

plt.show()


## 2. *Classical* Machine Learning: Random Forest Classifier

A random forest is a meta estimator that fits a number of classifying decision trees on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting.

In [ ]:
# The random forest expects a vector of features. Therefore,
# we concatenate all bands and pixels
X = np.reshape(sen2, (2400, 32 * 32 * 10))

print("Post-processed Sentinel-2 data shape: ", X_train.shape)

# Let us split the data into train and test
TRAIN_SPLIT = int(labels.shape[0] * .8)

X_train = X[:TRAIN_SPLIT,:]
X_test = X[TRAIN_SPLIT:,:]



In [ ]:
# The labels are one hot encoded, but the random forest requires
# the class number
y_train = np.argmax(labels[:TRAIN_SPLIT,:], axis=1)
y_test = np.argmax(labels[TRAIN_SPLIT:,:], axis=1)

print("Post-processed train labels shape: ", y_train.shape)

In [ ]:
rf_classifier = RandomForestClassifier(random_state=0)

rf_classifier.fit(X_train, y_train)

In [ ]:
y_pred_rf = rf_classifier.predict(X_test)

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_test, y_pred_rf)
plt.show()

acc_rf = accuracy_score(y_test, y_pred_rf)
print("Accuracy Random Forest Classifier: ", acc_rf)

## 3. Deep Learning



In [ ]:
Z_train = sen2[:TRAIN_SPLIT,:,:,:]
Z_test = sen2[TRAIN_SPLIT:,:,:,:]

print("Train shape: ", M_train.shape, y_train.shape)

y_train_oh = labels[:TRAIN_SPLIT,:]

In [ ]:
simple_model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(32, 32, 10)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(17, activation='softmax')
])

simple_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
simple_model.fit(Z_train, y_train, epochs=5)

In [ ]:
y_pred_dl_p = simple_model.predict(Z_test)

print("Prediction example: ", y_pred_dl_p[0,:], " Class: ", np.argmax(y_pred_dl_p[0,:]))

In [ ]:
y_pred_dl = np.argmax(y_pred_dl_p, axis=1)

ConfusionMatrixDisplay.from_predictions(y_test, y_pred_dl)
plt.show()

acc_dl = accuracy_score(y_test, y_pred_dl)
print("Accuracy Simple Deep Learning model: ", acc_dl)

Let us design a *deeper* model, based on ResNet... We will use Keras [implementation](https://github.com/keras-team/keras-applications/blob/master/keras_applications/resnet50.pyhttps://github.com/keras-team/keras-applications/blob/master/keras_applications/resnet50.py) as our basis

In [ ]:
def identity_block(
    input_tensor, 
    kernel_size, 
    filters, 
    stage,
    block):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of
            middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    input_tensor_shortcut = input_tensor
    
    # First component of main path
    input_tensor = tf.keras.layers.Conv2D(
        filters=F1, 
        kernel_size=(1, 1), 
        strides=(1,1), 
        padding='valid', 
        name=conv_name_base + '2a', 
        kernel_initializer=tf.keras.initializers.GlorotUniform(seed=0))(input_tensor)
    input_tensor = tf.keras.layers.BatchNormalization(
        axis=3, 
        name=bn_name_base + '2a')(input_tensor)
    input_tensor = tf.keras.layers.Activation('relu')(input_tensor)
    
    # Second component of main path
    input_tensor=tf.keras.layers.Conv2D(
        filters=F2, 
        kernel_size=kernel_size, 
        strides=(1,1), 
        padding='same', 
        name=conv_name_base + '2b', 
        kernel_initializer=tf.keras.initializers.GlorotUniform(seed=0))(input_tensor)
    input_tensor = tf.keras.layers.BatchNormalization(
        axis=3, 
        name=bn_name_base + '2b')(input_tensor)
    input_tensor = tf.keras.layers.Activation('relu')(input_tensor)

    # Third component of main path (≈2 lines)
    input_tensor = tf.keras.layers.Conv2D(
        filters=F3, 
        kernel_size=(1, 1), 
        strides=(1,1), 
        padding='valid', 
        name=conv_name_base + '2c', 
        kernel_initializer=tf.keras.initializers.GlorotUniform(seed=0))(input_tensor)
    input_tensor = tf.keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '2c')(input_tensor)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    input_tensor = tf.keras.layers.Add()([input_tensor, input_tensor_shortcut])
    input_tensor = tf.keras.layers.Activation('relu')(input_tensor)  
    
    return input_tensor

In [ ]:
def convolutional_block(
    input_tensor, 
    kernel_size, 
    filters, 
    stage, 
    block, 
    strides=(2, 2)):
    """A block that has a conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of
            middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
        strides: Strides for the first conv layer in the block.
    # Returns
        Output tensor for the block.
    """    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    input_tensor_shortcut = input_tensor


    # First component of main path 
    input_tensor = tf.keras.layers.Conv2D(
        F1, 
        (1, 1), 
        strides = strides, 
        name = conv_name_base + '2a', 
        kernel_initializer = tf.keras.initializers.GlorotUniform(seed=0))(input_tensor)
    input_tensor = tf.keras.layers.BatchNormalization(
        axis = 3, 
        name = bn_name_base + '2a')(input_tensor)
    input_tensor = tf.keras.layers.Activation('relu')(input_tensor)

    input_tensor = tf.keras.layers.Conv2D(
        filters = F2, 
        kernel_size = kernel_size, 
        strides = (1,1), 
        padding = 'same', 
        name = conv_name_base + '2b', 
        kernel_initializer = tf.keras.initializers.GlorotUniform(seed=0))(input_tensor)
    input_tensor = tf.keras.layers.BatchNormalization(
        axis = 3, 
        name = bn_name_base + '2b')(input_tensor)
    input_tensor = tf.keras.layers.Activation('relu')(input_tensor)

    input_tensor = tf.keras.layers.Conv2D(
        filters = F3, 
        kernel_size = (1, 1), 
        strides = (1,1), 
        padding = 'valid', 
        name = conv_name_base + '2c', 
        kernel_initializer = tf.keras.initializers.GlorotUniform(seed=0))(input_tensor)
    input_tensor = tf.keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '2c')(input_tensor)

    input_tensor_shortcut = tf.keras.layers.Conv2D(
        filters = F3, 
        kernel_size = (1, 1), 
        strides = strides, 
        padding = 'valid', 
        name = conv_name_base + '1',
        kernel_initializer = tf.keras.initializers.GlorotUniform(seed=0))(input_tensor_shortcut)
    input_tensor_shortcut = tf.keras.layers.BatchNormalization(
        axis = 3, 
        name = bn_name_base + '1')(input_tensor_shortcut)
    
    input_tensor = tf.keras.layers.Add()([input_tensor, input_tensor_shortcut])
    input_tensor = tf.keras.layers.Activation('relu')(input_tensor)
        
    return input_tensor

In [ ]:
def ResNet50(input_shape=(32, 32, 10), classes=17):
    # Define the input of the model
    M_input = tf.keras.layers.Input(input_shape)
    print("Input shape", M_input.shape)

    # Add zero padding to the patch
    M = tf.keras.layers.ZeroPadding2D(padding=(3, 3))(M_input)
    # Stage 1
    M = tf.keras.layers.Conv2D(
        filters=64, 
        kernel_size=(7, 7), 
        strides=(2, 2), 
        name='conv1', 
        kernel_initializer=tf.keras.initializers.GlorotUniform(seed=0))(M)
    M = tf.keras.layers.BatchNormalization(
        axis=3, 
        name='bn_conv1')(M)
    M = tf.keras.layers.Activation('relu')(M)
    M = tf.keras.layers.MaxPool2D(
        pool_size=(3, 3), 
        strides=(2, 2))(M)
    print("Stage 1 shape", M.shape)

    # Stage 2
    M = convolutional_block(
        M, 
        kernel_size=3, 
        filters=[32, 32, 256], 
        stage=2, 
        block='a', 
        strides=(1, 1))
    M = identity_block(M, 3, [64, 64, 256], stage=2, block='b')
    M = identity_block(M, 3, [64, 64, 256], stage=2, block='c')
    print("Stage 2 shape", M.shape)

    # Stage 3
    M = convolutional_block(
        M, 
        kernel_size=3, 
        filters=[128, 128, 512], 
        stage = 3, 
        block='a', 
        strides=(1, 1))
    M = identity_block(M, 3, [128, 128, 512], stage=3, block='b')
    M = identity_block(M, 3, [128, 128, 512], stage=3, block='c')
    M = identity_block(M, 3, [128, 128, 512], stage=3, block='d')
    print("Stage 3 shape", M.shape)
    
    # Stage 4
    M = convolutional_block(
        M, 
        kernel_size=3, 
        filters=[256, 256, 1024], 
        stage = 4, 
        block='a', 
        strides=(2, 2))
    M = identity_block(M, 3, [256, 256, 1024], stage=4, block='b')
    M = identity_block(M, 3, [256, 256, 1024], stage=4, block='c')
    M = identity_block(M, 3, [256, 256, 1024], stage=4, block='d')
    M = identity_block(M, 3, [256, 256, 1024], stage=4, block='e')
    M = identity_block(M, 3, [256, 256, 1024], stage=4, block='f')
    print("Stage 4 shape", M.shape)

    # Stage 5
    M = convolutional_block(
        M, 
        kernel_size=3, 
        filters=[512, 512, 2048], 
        stage = 5, 
        block='a', 
        strides=(2, 2))
    M = identity_block(M, 3, [512, 512, 2048], stage=5, block='b')
    M = identity_block(M, 3, [512, 512, 2048], stage=5, block='c')
    print("Stage 5 shape", M.shape)

    # AVGPOOL
    M = tf.keras.layers.AveragePooling2D((2,2), name="avg_pool")(M)
    print("Avg pool shape", M.shape)
    
    # output layer
    M = tf.keras.layers.Flatten()(M)
    M = tf.keras.layers.Dense(
        classes, 
        activation='softmax', 
        name='fc' + str(classes), 
        kernel_initializer = tf.keras.initializers.GlorotUniform)(M)
    print("Output shape", M.shape)
    
    # Create model
    model = tf.keras.Model(inputs = M_input, outputs = M, name='ResNet50')
    return model

In [ ]:
model_rn = ResNet50()


model_rn.compile(
    optimizer='adam', 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])

In [ ]:
model_rn.fit(Z_train, y_train, epochs=5)

In [ ]:
y_pred_rn_p = model_rn.predict(Z_test)

y_pred_rn = np.argmax(y_pred_rn_p, axis=1)

ConfusionMatrixDisplay.from_predictions(y_test, y_pred_rn)
plt.show()

acc_rn = accuracy_score(y_test, y_pred_rn)
print("Accuracy ResNet-50 model: ", acc_rn)

## 4. Open Challenges